<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span><ul class="toc-item"><li><span><a href="#Presets" data-toc-modified-id="Presets-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Presets</a></span></li><li><span><a href="#Style,-imports,-and-data" data-toc-modified-id="Style,-imports,-and-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Style, imports, and data</a></span></li><li><span><a href="#GPU" data-toc-modified-id="GPU-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>GPU</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Modeling-with-Time-Series-Generator" data-toc-modified-id="Modeling-with-Time-Series-Generator-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Modeling with Time Series Generator</a></span></li></ul></li></ul></div>

# Intro

## Presets

In [1]:
USE_GPU = True
IGNORE_WARN = True
SEED = 42

## Style, imports, and data

In [2]:
import warnings
import matplotlib as mpl
mpl.rcParams['axes.titlesize']='xx-large'
mpl.rcParams['xtick.labelsize'] = 13
mpl.rcParams['ytick.labelsize'] = 13
mpl.rcParams['axes.labelsize'] = 16
if IGNORE_WARN:
    warnings.filterwarnings('ignore')

In [3]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

In [5]:
import importlib
import sys

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding
# from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.preprocessing import sequence

## GPU

In [7]:
if USE_GPU:
    # Enable GPU
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
    os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

    # Show GPU
    print("Using GPU")
    print(tf.config.list_physical_devices('GPU'))
else:
    print("Using CPU")

Using GPU
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Model

## Modeling with Time Series Generator

In [8]:
test_df = pd.DataFrame({
    'date': [1, 2, 3, 4, 5],
    'orange': [1, 5, 3, 4, 2],
    'banana': [2, 4, 7, 4, 5],
    'apple': [4, 2, 2, 5, 8],
    'pear': [2, 7, 5, 4, 2]
    })
test_df = test_df.set_index('date', drop=True)
test_df.head()

,orange,banana,apple,pear
date,,,,
1,1,2,4,2
2,5,4,2,7
3,3,7,2,5
4,4,4,5,4
5,2,5,8,2


In [10]:
def contains(sub, pri):
    "https://stackoverflow.com/questions/3847386/how-to-test-if-a-list-contains-another-list" 
    M, N = len(pri), len(sub)
    i, LAST = 0, M-N+1
    while True:
        try:
            found = pri.index(sub[0], i, LAST) # find first elem in sub
        except ValueError:
            return False
        if pri[found:found+N] == sub:
            return [found, found+N-1]
        else:
            i = found+1
            
#contains(y_cols, X_cols)

In [11]:


def model_network(df, model, X_cols, y_cols, n_input):
    
    
    try:
        if X_cols == y_cols:
            print('y is the same as x')
            df = df[X_cols]
        elif ((contains(y_cols, X_cols)[1]
            - contains(y_cols, X_cols)[0])
            + 1 == len(y_cols)):
            print("y is in x")
        else:
            print('y is different than x')
            select_cols = X_cols + y_cols
            df = df[select_cols]
    except TypeError:
        print('y is in dataframe but not x')
    
    df_train = df.copy()
    
    # Account for another scaler
    if X_cols is not y_cols:
        X_df_train = df_train[X_cols]
        y_df_train = df_train[y_cols]
        
        X_scaler = MinMaxScaler()
        y_scaler = MinMaxScaler()
        
        X_train = X_scaler.fit_transform(X_df_train)
        y_train = y_scaler.fit_transform(y_df_train)

    
    # If X and y are the same i.e predicting self with self
    else:
        X_scaler = MinMaxScaler()
        y_scaler = 0
        df_train_scaled = X_scaler.fit_transform(df_train)
        X_train = df_train_scaled.copy()
        y_train = df_train_scaled.copy()

    # Reshape
    X_n_features = X_train.shape[1]
    y_n_features = y_train.shape[1]
    
    X_train_reshaped = X_train.reshape((len(X_train), X_n_features))
    y_train_reshaped = y_train.reshape((len(y_train), y_n_features))
    
    train_data_gen = sequence.TimeseriesGenerator( \
                         X_train_reshaped,
                         y_train_reshaped,#[:,column_indices[y_cols]],
                         length=n_input)
    input_shape = (X_train_reshaped.shape[0], X_train_reshaped.shape[1])
    
    history = model.fit(train_data_gen, epochs=10)
    
    
X_cols = list(test_df.columns)
y_cols = ['apple', 'pear']
n_features = len(y_cols)
n_days = 1
split_point = 2
input_shape = (2, len(X_cols))
try:
    del model
except NameError:
    pass

model = Sequential()
model.add(LSTM(64, input_shape=input_shape))
model.add(Dropout(.3))
for i in range(1):
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.1))
model.add(Dense(len(y_cols)))

model.compile(optimizer='adam', loss='mse')
    

model_network(test_df, model,  ['apple', 'banana', 'pear'], ['orange'], 1)

y is in dataframe but not x
Epoch 1/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4667
Epoch 2/10
1/1 [==============================] - 0s 995us/step - loss: 0.4467
Epoch 3/10
1/1 [==============================] - 0s 1000us/step - loss: 0.4435
Epoch 4/10
1/1 [==============================] - 0s 1ms/step - loss: 0.4358
Epoch 5/10
1/1 [==============================] - 0s 0s/step - loss: 0.4334
Epoch 6/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4172
Epoch 7/10
1/1 [==============================] - 0s 998us/step - loss: 0.4186
Epoch 8/10
1/1 [==============================] - 0s 999us/step - loss: 0.4014
Epoch 9/10
1/1 [==============================] - 0s 1000us/step - loss: 0.3959
Epoch 10/10
1/1 [==============================] - 0s 995us/step - loss: 0.3838


In [ ]:
df = df_hermes.copy()
df_train = df.iloc[:250].copy()
df_test = df.iloc[250:-20].copy()
df_val = df.iloc[-20::].copy()

X_df_train = df_train[X_cols]
y_df_train = df_train[y_cols]
X_df_test = df_test[X_cols]
y_df_test = df_test[y_cols]
X_df_val = df_val[X_cols]
y_df_val = df_val[y_cols]
print(y_df_train.shape)
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_train = X_scaler.fit_transform(X_df_train)
y_train = y_scaler.fit_transform(y_df_train)
X_test = X_scaler.transform(X_df_test)
y_test = y_scaler.transform(y_df_test)
X_val = X_scaler.transform(X_df_val)
y_val = y_scaler.transform(y_df_val)


X_train = X_scaler.fit_transform(X_df_train)
y_train = y_scaler.fit_transform(y_df_train)
y_n_features = y_train.shape[1]
y_n_features

In [16]:
def model_network(df, model, X_cols, y_cols, n_input):
    print(df.shape, len(X_cols), len(y_cols))
    if isinstance(X_cols, str):
        X_cols = [col for col in df.columns if X_cols in col]
    if isinstance(y_cols, str):
        y_cols = [col for col in df.columns if y_cols in col]
    
    try:
        if X_cols == y_cols:
            print('y is the same as x')
            df = df[X_cols]
        elif ((contains(y_cols, X_cols)[1]
            - contains(y_cols, X_cols)[0])
            + 1 == len(y_cols)):
            print("y is in x")
        else:
            print('y is different than x')
            select_cols = X_cols + y_cols
            df = df[select_cols]
    except TypeError:
        print('y is in dataframe but not x')
        
    print("X_cols", len(X_cols))
    print("y_cols", len(y_cols))

#     if X_cols == y_cols:
#         print('y is the same as x')
#         df = df[X_cols]
#     elif contains(y_cols, X_cols)[1] - contains(y_cols, X_cols)[0] + 1 == len(y_cols):
#         print("y is in x")
#     else:
#         print('y is different than x')
#         select_cols = X_cols + y_cols
#         df = df[select_cols]
#         display(df.columns)
    #column_indices = {name: i for i, name in enumerate(df.columns)}

    # Split dataframes
    df_train = df.iloc[:250].copy()
    df_test = df.iloc[250:-20].copy()
    df_val = df.iloc[-20::].copy()

    # Account for another scaler
    if X_cols is not y_cols:
        print("Creating y scaler")
        X_df_train = df_train[X_cols]
        y_df_train = df_train[y_cols]
        X_df_test = df_test[X_cols]
        y_df_test = df_test[y_cols]
        X_df_val = df_val[X_cols]
        y_df_val = df_val[y_cols]

        X_scaler = MinMaxScaler()
        y_scaler = MinMaxScaler()

        X_train = X_scaler.fit_transform(X_df_train)
        y_train = y_scaler.fit_transform(y_df_train)
        X_test = X_scaler.transform(X_df_test)
        y_test = y_scaler.transform(y_df_test)
        X_val = X_scaler.transform(X_df_val)
        y_val = y_scaler.transform(y_df_val)

    # If X and y are the same i.e predicting self with self
    else:
        X_scaler = MinMaxScaler()
        y_scaler = 0
        df_train_scaled = X_scaler.fit_transform(df_train)
        df_test_scaled = X_scaler.transform(df_test)
        df_val_scaled = X_scaler.transform(df_val)
        X_train = df_train_scaled.copy()
        y_train = df_train_scaled.copy()
        X_test = df_test_scaled.copy()
        y_test = df_test_scaled.copy()
        X_val = df_val_scaled.copy()
        y_val = df_val_scaled.copy()

    # Get n_features
    X_n_features = X_train.shape[1]
    y_n_features = y_train.shape[1]
    print(y_train.shape)
    
    print("len(X_train)", len(X_train))
    print("len(y_train)", len(y_train))

    # Reshape data
    X_train_reshaped = X_train.reshape((len(X_train), X_n_features))
    y_train_reshaped = y_train.reshape((len(y_train), y_n_features))
    X_test_reshaped = X_test.reshape((len(X_test), X_n_features))
    y_test_reshaped = y_test.reshape((len(y_test), y_n_features))
    X_val_reshaped = X_val.reshape((len(X_val), X_n_features))
    y_val_reshaped = y_val.reshape((len(y_val), y_n_features))

    # Get data generators
    print("create generator")
    import time
    old = time.time()
    train_data_gen = sequence.TimeseriesGenerator( \
                         X_train_reshaped,
                         y_train_reshaped,#[:,column_indices[y_cols]],
                         length=n_input)
    test_data_gen = sequence.TimeseriesGenerator( \
                        X_test_reshaped,
                        y_test_reshaped,#[:,column_indices[y_cols]],
                        length=n_input)
    val_data_gen = sequence.TimeseriesGenerator( \
                        X_val_reshaped,
                        y_val_reshaped,#[:,column_indices[y_cols]],
                        length=n_input)
    print("created in %ss" % (time.time()-old))

    input_shape = (X_train_reshaped.shape[0], X_train_reshaped.shape[1])
    print("input_shape", input_shape)
    
    print("X_n_features", X_n_features)
    print("y_n_features", y_n_features)
    # Model
    model.compile(optimizer='adam', loss='mse')

    earlystopping = EarlyStopping(monitor='val_loss', patience=25)
    history = model.fit(train_data_gen,
                        epochs=2000, batch_size=64,
                        validation_data=(test_data_gen),
                        verbose=2, shuffle=False,
                        callbacks=[earlystopping])

    # my_loss= history.history['loss']
    # plt.plot(range(len(my_loss)),my_loss)


    # Predictions (r2_score)

    # If predicting one column for y
    if y_scaler and (len(y_cols) == 1):
        df_train['predicted'] = pd.DataFrame(y_scaler.inverse_transform(model.predict(train_data_gen)), columns=y_cols,
                                            index=df_train[n_input:][y_cols].index)
        df_test['predicted'] = pd.DataFrame(y_scaler.inverse_transform(model.predict(test_data_gen)), columns=y_cols,
                                            index=df_test[n_input:][y_cols].index)
        df_val['predicted'] = pd.DataFrame(y_scaler.inverse_transform(model.predict(val_data_gen)), columns=y_cols,
                                            index=df_val[n_input:][y_cols].index)
        cols = y_cols + ['predicted']
        df_predict_train = df_train[cols].dropna()
        df_predict_test = df_test[cols].dropna()
        df_predict_val = df_val[cols].dropna()

        train_r2 = r2_score(df_predict_train[y_cols], df_predict_train[['predicted']])
        test_r2 = r2_score(df_predict_test[y_cols], df_predict_test[['predicted']])
        val_r2 = r2_score(df_predict_val[y_cols], df_predict_val[['predicted']])

        print("Train r2 =", train_r2)
        print("Test r2 =", test_r2)
        print("Val r2 =", val_r2)


        fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(15, 6))
        df_predict_train[cols].plot(ax=ax1)
        df_predict_test[cols].plot(ax=ax2)
        df_predict_val[cols].plot(ax=ax3)

    # If predicting multiple columns for y
    elif y_scaler:
        df_predict_train = pd.DataFrame(y_scaler.inverse_transform(model.predict(train_data_gen)), columns=y_cols,
                                            index=y_df_train[n_input:].index)
        df_predict_test = pd.DataFrame(y_scaler.inverse_transform(model.predict(test_data_gen)), columns=y_cols,
                                            index=y_df_test[n_input:].index)
        df_predict_val = pd.DataFrame(y_scaler.inverse_transform(model.predict(val_data_gen)), columns=y_cols,
                                            index=y_df_val[n_input:].index)

        y_true_train = y_df_train.loc[df_predict_train.index]
        y_true_test = y_df_test.loc[df_predict_test.index]
        y_true_val = y_df_val.loc[df_predict_val.index]

        train_r2 = r2_score(y_true_train, df_predict_train)
        test_r2 = r2_score(y_true_test, df_predict_test)
        val_r2 = r2_score(y_true_val, df_predict_val)
        print("Train r2 =", train_r2)
        print("Test r2 =", test_r2)
        print("Val r2 =", val_r2)

    # if predicting self with self
    else:
        df_predict_train = pd.DataFrame(X_scaler.inverse_transform(model.predict(train_data_gen)), columns=y_cols,
                                         index=df_train[n_input:].index)
        df_predict_test = pd.DataFrame(X_scaler.inverse_transform(model.predict(test_data_gen)), columns=y_cols,
                                         index=df_test[n_input:].index)
        df_predict_val = pd.DataFrame(X_scaler.inverse_transform(model.predict(val_data_gen)), columns=y_cols,
                                         index=df_val[n_input:].index)

        y_true_train = df_train.loc[df_predict_train.index]
        y_true_test = df_test.loc[df_predict_test.index]
        y_true_val = df_val.loc[df_predict_val.index]

        train_r2 = r2_score(y_true_train, df_predict_train)
        test_r2 = r2_score(y_true_test, df_predict_test)
        val_r2 = r2_score(y_true_val, df_predict_val)
        print("Train r2 =", train_r2)
        print("Test r2 =", test_r2)
        print("Val r2 =", val_r2)

In [24]:
df_hermes.shape, len(X_cols), len(y_cols)

((342, 87462), 81270, 12384)

In [18]:
df_analyst.shape

(342, 6192)

In [19]:
input_shape

(250, 81270)

In [22]:
df_hermes.shape, df_analyst.shape

((342, 87462), (342, 6192))

In [21]:
df_total.shape

(342, 93654)

In [37]:
len([col for col in df_hermes.columns if 'Zacks' in col])

387

In [39]:
df_hermes = pd.read_pickle("./data/modeling/hermes.pkl")
df_analyst = pd.read_pickle("./data/modeling/analyst.pkl")
#df_total = pd.concat([df_hermes, df_analyst], axis=1)
X_cols = [col for col in df_hermes.columns if col not in df_analyst.columns]
y_cols = [col for col in df_hermes.columns if col in df_analyst.columns]
n_days = 4
split_point = 250
input_shape = (split_point, len(X_cols))

model = Sequential()
model.add(LSTM(64, input_shape=input_shape))
model.add(Dropout(.3))
for i in range(1):
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.1))
model.add(Dense(len(y_cols)))

model.compile(optimizer='adam', loss='mse')

creator = model_network(df_hermes, model, X_cols, y_cols, n_days)


(342, 87462) 81270 6192
y is in dataframe but not x
X_cols 81270
y_cols 6192
Creating y scaler
(250, 6192)
len(X_train) 250
len(y_train) 250
create generator
created in 0.0s
input_shape (250, 81270)
X_n_features 81270
y_n_features 6192
Epoch 1/2000
2/2 - 1s - loss: 0.3369 - val_loss: 0.5805
Epoch 2/2000
2/2 - 0s - loss: 0.3138 - val_loss: 0.5493
Epoch 3/2000


KeyboardInterrupt: 

In [116]:
df_analyst = pd.read_pickle("./data/modeling/analyst.pkl")
df_prices = pd.read_pickle("./data/modeling/prices.pkl")

df_cronus = pd.concat([df_analyst, df_prices], axis=1)
X_cols = list(df_cronus.columns)
y_cols = list(df_prices.columns)
n_features = len(y_cols)
n_days = 1
split_point = 250
input_shape = (split_point, len(X_cols))
try:
    del model
except NameError:
    pass

model = Sequential()
model.add(LSTM(64, input_shape=input_shape))
model.add(Dropout(.3))
for i in range(1):
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.1))
model.add(Dense(len(y_cols)))

model.compile(optimizer='adam', loss='mse')


model_network(df_cronus, model, X_cols, y_cols, n_days)

here
len(X_train) 250
len(y_train) 250
create generator
created in 0.0s
input_shape (250, 6834)
X_n_features 6834
y_n_features 402
Epoch 1/2000
2/2 - 1s - loss: 0.4103 - val_loss: 0.5771
Epoch 2/2000
2/2 - 0s - loss: 0.3780 - val_loss: 0.5510
Epoch 3/2000
2/2 - 0s - loss: 0.3537 - val_loss: 0.5132
Epoch 4/2000
2/2 - 0s - loss: 0.3278 - val_loss: 0.4819
Epoch 5/2000
2/2 - 0s - loss: 0.3010 - val_loss: 0.4487
Epoch 6/2000
2/2 - 0s - loss: 0.2754 - val_loss: 0.4130
Epoch 7/2000
2/2 - 0s - loss: 0.2474 - val_loss: 0.3768
Epoch 8/2000
2/2 - 0s - loss: 0.2235 - val_loss: 0.3405
Epoch 9/2000
2/2 - 0s - loss: 0.1987 - val_loss: 0.3043
Epoch 10/2000
2/2 - 0s - loss: 0.1762 - val_loss: 0.2705
Epoch 11/2000
2/2 - 0s - loss: 0.1514 - val_loss: 0.2399
Epoch 12/2000
2/2 - 0s - loss: 0.1332 - val_loss: 0.2134
Epoch 13/2000
2/2 - 0s - loss: 0.1185 - val_loss: 0.1900
Epoch 14/2000
2/2 - 0s - loss: 0.1070 - val_loss: 0.1706
Epoch 15/2000
2/2 - 0s - loss: 0.0964 - val_loss: 0.1572
Epoch 16/2000
2/2 - 0s 

In [114]:
X_cols = list(df_cronus.columns)
y_cols = list(df_prices.columns)
contains(y_cols, X_cols)[1] - contains(y_cols, X_cols)[0] + 1

402

In [112]:
len(y_cols)

402

In [ ]:
# df = df[[col for col in df.columns if 'VLO' in col or 'TSLA' in col]]
# X_cols = [col for col in df.columns if 'price' not in col]
# y_cols = 'price'
#X_cols = [col for col in df.columns if 'TSLA' in col]
#y_cols = X_cols
#X_cols = [col for col in df.columns if 'TSLA' in col and 'price' not in col]

df = pd.read_pickle('./data/modeling/model_df.pkl')
# X_cols = [col for col in df.columns if 'TSLA_price' not in col]
# y_cols = X_cols
X_cols = [col for col in df.columns if 'TSLA' in col]
y_cols = X_cols
n_features = len(y_cols)
n_days = 1
df.head(1)

split_point = 250
input_shape = (split_point, len(X_cols))

try:
    del model
except NameError:
    pass

model = Sequential()
model.add(LSTM(64, input_shape=input_shape))
model.add(Dropout(.3))
for i in range(1):
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.1))
model.add(Dense(len(y_cols)))


model_network(df, model, X_cols, y_cols, n_days)